<a href="https://colab.research.google.com/github/TreasureSam/IBM-Granite/blob/main/notebooks/Summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Summarization


This notebook demonstrates an application of long document summarization techniques to a work of literature using Granite.

### Python version

Ensure you are running python 3.10, 3.11, or 3.12 in a freshly-created virtual environment.

In [5]:
import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 13), "Use Python 3.10, 3.11, or 3.12 to run this notebook."

## Install Dependencies


Granite utils provides some helpful functions for recipes.

In [6]:
! pip install git+https://github.com/ibm-granite-community/utils \
    langchain_community \
    transformers \
    langchain_huggingface \
    langchain_ollama \
    replicate \
    docling

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-0j3cmhqa
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-0j3cmhqa
  Resolved https://github.com/ibm-granite-community/utils to commit 157a188d350c6220679f40106a20d7c230e9b61d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

## Serving the Granite AI model


This notebook requires IBM Granite models to be served by an AI model runtime so that the models can be invoked or called. This notebook can use a locally accessible [Ollama](https://github.com/ollama/ollama) server to serve the models, or the [Replicate](https://replicate.com) cloud service.

During the pre-work, you may have either started a local Ollama server on your computer, or setup Replicate access and obtained an [API token](https://replicate.com/account/api-tokens).

## Select your model

Select a Granite model to use. Here we use a Langchain client to connect to the model. If there is a locally accessible Ollama server, we use an Ollama client to access the model. Otherwise, we use a Replicate client to access the model.

When using Replicate, if the `REPLICATE_API_TOKEN` environment variable is not set, or a `REPLICATE_API_TOKEN` Colab secret is not set, then the notebook will ask for your [Replicate API token](https://replicate.com/account/api-tokens) in a dialog box.

In [7]:
import os
import requests
from langchain_ollama.llms import OllamaLLM
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

model_path = "ibm-granite/granite-3.3-8b-instruct"
try: # Look for a locally accessible Ollama server for the model
    response = requests.get(os.getenv("OLLAMA_HOST", "http://127.0.0.1:11434"))
    model = OllamaLLM(
        model="granite3.3:8b",
        num_ctx=65536, # 64K context window
    )
    model = model.bind(raw=True) # Client side controls prompt
except Exception: # Use Replicate for the model
    model = Replicate(
        model=model_path,
        replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
        model_kwargs={
            "max_tokens": 2000, # Set the maximum number of tokens to generate as output.
            "min_tokens": 200, # Set the minimum number of tokens to generate as output.
            "temperature": 0.75,
            "presence_penalty": 0,
            "frequency_penalty": 0,
        },
    )

REPLICATE_API_TOKEN not found in Google Colab secrets.
Please enter your REPLICATE_API_TOKEN: ··········


## Split a book into chunks

The code in the following sections will fetch H.D. Thoreau's "Walden" from [Project Gutenberg](https://www.gutenberg.org/) for summarization.

We will then chunk the book text so that chunks fit in the context window size of the AI model.

### Count the tokens

Before sending our book chunks to the AI model, it's crucial to understand how much of the model's capacity we're using. Language models typically have a limit on the number of tokens they can process in a single request.

Key points:
- We're using the [`granite-3.3`](https://huggingface.co/ibm-granite/granite-3.3-2b-instruct) model, which has a context window of 128K tokens.
- Tokenization can vary between models, so we use the specific tokenizer for our chosen model.

Understanding token count helps us optimize our prompts and ensure we're using the model efficiently.

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

### Summary of Summaries

Here we use a hierarchical abstractive summarization technique to adapt to the context length of the model. Our approach uses [Docling](https://docling-project.github.io/docling/) to understand the document's structure, chunk the document into text passages, and group the text passages by chapter which we can then summarize.

In [9]:
import itertools
from typing import Iterator, Callable
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker.hierarchical_chunker import HierarchicalChunker
from docling_core.transforms.chunker.base import BaseChunk

def chunk_document(source: str, *, dropwhile: Callable[[BaseChunk], bool] = lambda c: False, takewhile: Callable[[BaseChunk], bool] = lambda c: True) -> Iterator[BaseChunk]:
    """Read the document and perform a hierarchical chunking"""
    converter = DocumentConverter()
    chunks = HierarchicalChunker().chunk(converter.convert(source=source).document)
    return itertools.takewhile(takewhile, itertools.dropwhile(dropwhile, chunks))

def merge_chunks(chunks: Iterator[BaseChunk], *, headings: Callable[[BaseChunk], list[str]] = lambda c: c.meta.headings) -> Iterator[dict[str, str]]:
    """Merge chunks having the same headings"""
    prior_headings: list[str] | None = None
    document: dict[str, str] = {}
    doc_id = 0
    for chunk in chunks:
        text = chunk.text.replace('\r\n', '\n')
        current_headings = headings(chunk)
        if prior_headings != current_headings:
            if document:
                yield document
            prior_headings = current_headings
            document = {
                'doc_id': str(doc_id := doc_id + 1),
                'title': " - ".join(current_headings),
                'text': text
            }
        else:
            document['text'] += f"\n\n{text}"
    if document:
        yield document

def chunk_dropwhile(chunk: BaseChunk) -> bool:
    """Ignore front matter prior to the book start"""
    return "WALDEN" not in chunk.meta.headings

def chunk_takewhile(chunk: BaseChunk) -> bool:
    """Ignore remaining chunks once we see this heading"""
    return "ON THE DUTY OF CIVIL DISOBEDIENCE" not in chunk.meta.headings

def chunk_headings(chunk: BaseChunk) -> list[str]:
    """Use the h1 and h2 (chapter) headings"""
    return chunk.meta.headings[:2]

documents: list[dict[str, str]] = list(merge_chunks(
    chunk_document(
        "https://www.gutenberg.org/cache/epub/205/pg205-images.html",
        dropwhile=chunk_dropwhile,
        takewhile=chunk_takewhile,
    ),
    headings=chunk_headings,
))

print(f"{len(documents)} documents created")
print(f"Max document size: {max(len(tokenizer.tokenize(document['text'])) for document in documents)} tokens")

18 documents created
Max document size: 38299 tokens


## Summarize the chunks

Here we define a method to generate a response using a list of documents and a user prompt about those documents.

We create the prompt according to the [Granite Prompting Guide](https://www.ibm.com/granite/docs/models/granite/#basic-chat-template-example) and provide the documents using the `documents` parameter.

In [10]:
def generate(user_prompt: str, documents: list[dict[str, str]]):
    """Use the chat template to format the prompt"""
    prompt = tokenizer.apply_chat_template(
        conversation=[{
            "role": "user",
            "content": user_prompt,
        }],
        documents=documents, # This uses the documents support in the Granite chat template
        add_generation_prompt=True,
        tokenize=False,
    )

    print(f"Input size: {len(tokenizer.tokenize(prompt))} tokens")
    output = model.invoke(prompt)
    print(f"Output size: {len(tokenizer.tokenize(output))} tokens")

    return output


For each chapter, we create a separate summary. This can take a few minutes.

In [11]:
if get_env_var('GRANITE_TESTING', 'false').lower() == 'true':
    documents = documents[:5] # shorten testing work

user_prompt = """\
Using only the the book chapter document, compose a summary of the book chapter.
Your response should only include the summary. Do not provide any further explanation."""

summaries: list[dict[str, str]] = []

for i, document in enumerate(documents):
    print(f"============================= {document['title']} ({i+1}/{len(documents)}) =============================")
    output = generate(user_prompt, [document])
    summaries.append({
        'doc_id': document['doc_id'],
        'title': document['title'],
        'text': output,
    })

print("Summary count: " + str(len(summaries)))


GRANITE_TESTING not found in Google Colab secrets.
============================= WALDEN - Economy (1/18) =============================
Input size: 38447 tokens
Output size: 230 tokens
============================= WALDEN - Where I Lived, and What I Lived For (2/18) =============================
Input size: 9138 tokens
Output size: 212 tokens
============================= WALDEN - Reading (3/18) =============================
Input size: 5724 tokens
Output size: 208 tokens
============================= WALDEN - Sounds (4/18) =============================
Input size: 9107 tokens
Output size: 321 tokens
============================= WALDEN - Solitude (5/18) =============================
Input size: 5219 tokens
Output size: 214 tokens
============================= WALDEN - Visitors (6/18) =============================
Input size: 7153 tokens
Output size: 210 tokens
============================= WALDEN - The Bean-Field (7/18) =============================
Input size: 6345 tokens
Output size:

## Create the Final Summary

Now we need to summarize the chapter summaries. We prompt the model to create a unified summary of the chapter summaries we previously generated.

In [12]:
from ibm_granite_community.notebook_utils import wrap_text

user_prompt = """\
Using only the book chapter summary documents, compose a single, unified summary of the book.
Your response should only include the unified summary. Do not provide any further explanation."""

output = generate(user_prompt, summaries)
print(wrap_text(output))

Input size: 4513 tokens
Output size: 268 tokens
This book is a reflection on simple living and harmony with nature, as
experienced by the author in his secluded cabin near Walden Pond, Massachusetts.
The author values solitude, direct experience, and authenticity over material
possessions and superficial social interactions. He critiques modern
architecture's extravagance, advocating for modest, nature-inspired designs.
Gardening and farming are seen as essential connections to the natural world,
promoting self-reliance and spiritual growth. The author emphasizes the
importance of reading classic literature and ancient texts for intellectual and
spiritual development, criticizing the prevalence of easy reading culture. He
also values the natural rhythm of time and the spiritual upliftment derived from
a simple life, contrasting it with the hurried existence of many in society. The
text underscores the significance of living deliberately, striving for
authenticity, and valuing experienc

So we have now summarized a document larger than the AI model's context window length by breaking the document down into smaller pieces to summarize and then summarizing those summaries.